In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from google.colab import auth
from google.cloud import bigquery


In [ ]:
auth.authenticate_user()

project_id = 'googlebigqueryprojectid'
client = bigquery.Client(project=project_id)


In [ ]:
!pip install copernicusmarine

# Step 1: Install the Copernicus Marine Toolbox
import copernicusmarine

# Step 2: Login with Your Copernicus Marine Credentials
copernicusmarine.login(username="emailid", password="password")


In [ ]:
# Subset

copernicusmarine.subset(
   dataset_id = "METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2",
   variables = ["analysed_sst"],
   start_datetime = "2011-01-01T00:00:00",
   end_datetime = "2024-12-31T23:59:59",
   minimum_longitude = -59.75,
   maximum_longitude = -59.40,
   minimum_latitude = 12.95,
   maximum_latitude = 13.25,
   minimum_depth = 0,
   maximum_depth = 1
)

In [ ]:
import xarray as xr

# Open the NetCDF file
ds = xr.open_dataset('/content/METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_analysed_sst_59.72W-59.42W_12.98N-13.23N_2011-01-01-2024-12-31.nc')
print(ds)

In [ ]:
# Flatten and convert to dataframe
df = ds['analysed_sst'].to_dataframe(name='sea_surface_temp').reset_index()

In [ ]:
df.head()

,time,latitude,longitude,sea_surface_temp
0,2011-01-01,12.975,-59.724998,300.899993
1,2011-01-01,12.975,-59.674999,300.899993
2,2011-01-01,12.975,-59.625000,300.899993
3,2011-01-01,12.975,-59.575001,300.889993
4,2011-01-01,12.975,-59.525002,300.889993


In [ ]:
df.shape

(214788, 4)

In [ ]:
import numpy as np

# Load coordinate arrays from dataset
ds = xr.open_dataset("/content/METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_analysed_sst_59.72W-59.42W_12.98N-13.23N_2011-01-01-2024-12-31.nc")
latitudes = ds.latitude.values
longitudes = ds.longitude.values

# Known coastal coordinates (approximate)
locations = {
    "Crane Beach": (13.1089, -59.4414),
    # "Folkestone Park": (13.1912, -59.6423),
    "Oistins": (13.0647, -59.5515),
    "Conset Bay": (13.2025, -59.4827)
}

# Find nearest available offshore grid point for each location
offshore_matches = {}

for name, (lat, lon) in locations.items():
    closest_lat = latitudes[np.abs(latitudes - lat).argmin()]
    closest_lon = longitudes[np.abs(longitudes - lon).argmin()]
    offshore_matches[name] = (closest_lat, closest_lon)

offshore_matches


{'Crane Beach': (np.float32(13.125), np.float32(-59.425)),
 'Oistins': (np.float32(13.075), np.float32(-59.575)),
 'Conset Bay': (np.float32(13.225), np.float32(-59.475))}

In [ ]:
# Extract SST for each offshore location
location_sst = {}

for name, (lat_val, lon_val) in offshore_matches.items():
    sst = ds['analysed_sst'].sel(latitude=lat_val, longitude=lon_val).to_dataframe().reset_index()
    sst['location'] = name  # Add location column
    location_sst[name] = sst

# Combine all locations into one DataFrame
import pandas as pd
df_combined = pd.concat(location_sst.values(), ignore_index=True)

# Preview
df_combined.head()


,time,latitude,longitude,analysed_sst,location
0,2011-01-01,13.125,-59.424999,300.789993,Crane Beach
1,2011-01-02,13.125,-59.424999,300.769993,Crane Beach
2,2011-01-03,13.125,-59.424999,300.699993,Crane Beach
3,2011-01-04,13.125,-59.424999,300.869993,Crane Beach
4,2011-01-05,13.125,-59.424999,300.889993,Crane Beach


In [ ]:
from google.cloud import bigquery

# Set BigQuery config
project_id = "googleprojectid"
dataset_id = "sst_subset_barbados"
table_id = "major_locations"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Upload to BigQuery
client = bigquery.Client()
job = client.load_table_from_dataframe(df_combined, full_table_id)
job.result()  # Wait for it to finish

print(f"✅ Uploaded {len(df)} rows to {full_table_id}")
